## Set up area

In [20]:
# define library
import folium

# define coordinates
main_coords = [-8.3304977, 115.0906401]  # Bali coordinates
radius = 80000  # radius in meter

main_map = folium.Map(main_coords, zoom_start=9)

folium.Circle(main_coords, radius=radius, color='#ff0000', fill=True, fill_color='#ffff00', fill_opacity=0.2
            ).add_to(main_map)

main_map

## Scraping data

In [52]:
# libraries
import pandas as pd
import requests
import json

# API url
overpass_api = "http://overpass-api.de/api/interpreter"

# query
query_medical = """
[out:json];
area['name:id'='Provinsi Bali'];
node[amenity~'(clinic|dentist|doctors|hospital|pharmacy)'](area);
out;
"""

response_medical = requests.get(overpass_api, params={'data':query_medical})
data_medical = response_medical.json()

In [ ]:
# show the output
data_medical

In [53]:
list_medical = data_medical['elements']

In [ ]:
list_medical

In [54]:
df_medical_raw = pd.DataFrame.from_dict(pd.json_normalize(list_medical), orient='columns')

In [55]:
df_medical_raw.head()

,type,id,lat,lon,tags.addr:city,tags.addr:postcode,tags.addr:street,tags.amenity,tags.healthcare,tags.is_in,...,tags.website,tags.healthcare:speciality,tags.email,tags.name:de,tags.name:es,tags.name:zh,tags.highway,tags.addr:district,tags.name:fr,tags.internet_access
0,node,429250405,-8.354431,114.621454,Jembrana,82218,Wijaya Kusuma,hospital,hospital,"West Bali,Bali,Indonesia",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,600488283,-8.760220,115.175837,"Badung, Bali",80361,Jalan Uluwatu,hospital,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,964939213,-8.712574,115.173211,NaN,NaN,NaN,pharmacy,pharmacy,"Legian,South Bali,Bali,Indonesia",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,965148761,-8.717328,115.174146,NaN,NaN,NaN,pharmacy,NaN,"Kuta,South Bali,Bali,Indonesia",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,1067112056,-8.736851,115.167952,NaN,NaN,NaN,hospital,NaN,"South Bali,Bali,Indonesia",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# take only columns needed, others are dropped. rename columns and sort.

df_medical = df_medical_raw[['lat', 'lon', 'tags.name', 'tags.amenity']]\
                .rename(columns={'tags.name':'name', 'tags.amenity':'type'})\
                .sort_values(by=['name']).reset_index()

In [57]:
df_medical

,index,lat,lon,name,type
0,158,-8.657657,115.147088,Apotek,pharmacy
1,14,-8.540324,115.126398,Apotek Karunia,pharmacy
2,84,-8.766855,115.178133,Apotek Kimia Farma By Pass Ngurah Rai,pharmacy
3,69,-8.617890,115.196020,Apotek Kimia Farma Cokroaminoto,pharmacy
4,20,-8.681152,115.198651,Apotek Kimia Farma Teuku Umar,pharmacy
...,...,...,...,...,...
203,202,-8.658662,115.139889,NaN,pharmacy
204,203,-8.122974,115.071388,NaN,hospital
205,204,-8.111922,115.089568,NaN,hospital
206,205,-8.508834,115.267787,NaN,doctors


In [58]:
# fill NaN value in column 'name' with value from column 'type'

df_medical.name.fillna(df_medical.type, inplace=True)

In [60]:
# remove column 'index'

df_medical = df_medical.drop(columns=['index'])

In [64]:
df_medical

,lat,lon,name,type
0,-8.657657,115.147088,Apotek,pharmacy
1,-8.540324,115.126398,Apotek Karunia,pharmacy
2,-8.766855,115.178133,Apotek Kimia Farma By Pass Ngurah Rai,pharmacy
3,-8.617890,115.196020,Apotek Kimia Farma Cokroaminoto,pharmacy
4,-8.681152,115.198651,Apotek Kimia Farma Teuku Umar,pharmacy
...,...,...,...,...
203,-8.658662,115.139889,pharmacy,pharmacy
204,-8.122974,115.071388,hospital,hospital
205,-8.111922,115.089568,hospital,hospital
206,-8.508834,115.267787,doctors,doctors


In [62]:
# save dataframe into csv file

df_medical.to_csv('list_medical.csv', sep=',')

## Visualise the data 

In [63]:
island_coords = [-8.455, 115.09]     # coordinates of the island area

island_map = folium.Map(island_coords, zoom_start=10)

for group_name, group in df_medical.groupby('type'):
    feature_group = folium.FeatureGroup(group_name)
    
    for row in group.itertuples():
        if row[4] == 'clinic':
            folium.Marker(location=[row.lat, row.lon], tooltip=row[3], 
                          icon=folium.Icon(color='purple', icon_color='white', prefix='fa', icon='medkit')
                         ).add_to(feature_group)        
        
        elif row[4] == 'dentist':
            folium.Marker(location=[row.lat, row.lon], tooltip=row[3], 
                          icon=folium.Icon(color='green', icon_color='white', prefix='fa', icon='medkit')
                         ).add_to(feature_group)
        
        elif row[4] == 'doctors':
            folium.Marker(location=[row.lat, row.lon], tooltip=row[3], 
                          icon=folium.Icon(color='darkblue', icon_color='white', prefix='fa', icon='medkit')
                         ).add_to(feature_group)
        elif row[4] == 'hospital':
            folium.Marker(location=[row.lat, row.lon], tooltip=row[3], 
                          icon=folium.Icon(color='red', icon_color='white', prefix='fa', icon='medkit')
                         ).add_to(feature_group)
        else:   # pharmacy
            folium.Marker(location=[row.lat, row.lon], tooltip=row[3], 
                          icon=folium.Icon(color='orange', icon_color='white', prefix='fa', icon='medkit')
                         ).add_to(feature_group)
    feature_group.add_to(island_map)
    
folium.LayerControl().add_to(island_map)

island_map

In [66]:
# save the map into html file

island_map.save(outfile='Bali_medical_facilities.html')